# Plot results of the target phase scan

In [ ]:
import sys
import os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
import proplot as pplt

from target_image_analysis import Image
from target_image_analysis import PIXEL_WIDTH
from target_image_analysis import process_image_array

sys.path.append('/Users/46h/Research/')
from accphys.tools import utils
from accphys.tools import plotting as myplt

In [ ]:
pplt.rc['axes.grid'] = False
pplt.rc['savefig.transparent'] = True
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'dusk_r'
savefig_kws = dict(dpi=300)

## Optics 

In [ ]:
phase_adv = np.loadtxt('_output/data/phase_adv.dat')

In [ ]:
fig, ax = pplt.subplots(figsize=(6, 2.25))
g1 = ax.plot(phase_adv[:, 0], marker='.')
g2 = ax.plot(phase_adv[:, 1], marker='.')
ax.legend([g1, g2], labels=[r"$\mu_x$", r"$\mu_y$"], loc=(0., 1.), framealpha=0.)
ax.format(ylabel='[rad]', xlabel='Scan index', ylim=(0., 2*np.pi))
plt.savefig('_output/figures/phase_adv.png', **savefig_kws)

In [ ]:
twiss_filenames = [f for f in utils.list_files('_output/data/') if 'twiss' in f]
twiss_filenames = sorted(twiss_filenames, key=lambda f: float(f.split('.')[0].split('_')[-1]))
twiss_list = [np.loadtxt(f) for f in twiss_filenames]

In [ ]:
fig, ax = pplt.subplots(figsize=(6, 2.25))
plot_kws = dict(alpha=0.1, marker=None)
colors = myplt.DEFAULT_COLORCYCLE

for twiss in twiss_list:
    positions, mux, muy, alpha_x, alpha_y, beta_x, beta_y, eps_x, eps_y = twiss.T
    ax.plot(positions, beta_x, color=colors[0], **plot_kws)
    ax.plot(positions, beta_y, color=colors[1], **plot_kws)
ax.format(ylabel='[m/rad]', ylim=(0., 100.), xlabel='Position [m]')

lines = [Line2D([0], [0], color=colors[0]), Line2D([0], [0], color=colors[1])]
ax.legend(lines, [r"$\beta_x$", r"$\beta_y$"], loc=(0., 1.), framealpha=0.)
# plt.savefig('_output/figures/beta_vs_pos.png', **savefig_kws)
plt.show()

In [ ]:
fig, ax = pplt.subplots(figsize=(6, 2.25))
for twiss in twiss_list:
    positions, mu_x, mu_y, alpha_x, alpha_y, beta_x, beta_y, eps_x, eps_y = twiss.T
    ax.plot(positions, mu_x, color=colors[0], **plot_kws)
    ax.plot(positions, mu_y, color=colors[1], **plot_kws)
ax.format(ylabel='[m/rad]', xlabel='Position [m]', ylim=(0., 2*np.pi))

lines = [Line2D([0], [0], color=colors[0]), Line2D([0], [0], color=colors[1])]
ax.legend(lines, [r"$\mu_x$", r"$\mu_y$"], loc=(0., 1.), framealpha=0.)
# plt.savefig('_output/figures/phase_adv_vs_pos.png', **savefig_kws)
plt.show()

In [ ]:
fields = pd.read_table('_output/data/fields.dat', sep=' ')
fields.head()

In [ ]:
fig, ax = pplt.subplots()
ax.format(ylabel='Field strength [T/m]', cycle='538', xlabel='Scan index')
for col in fields.columns[:-1]:
    ax.plot(fields.loc[:, col], marker='.')
ax.legend(ncols=1, loc='r')
# plt.savefig('_output/figures/fields.png', **savefig_kws)
plt.show()

## Beam image 

In [ ]:
image_arrays = np.loadtxt('_output/data/images.dat')
images = []
for image_array in image_arrays:
    Z = process_image_array(image_array, make_square=False)
    images.append(Image(Z, pixel_width=PIXEL_WIDTH))

Check if the images are different.

In [ ]:
for i in range(len(images) - 1):
    print(i, np.all(images[i].Z == images[i + 1]))

Plot the images.

In [ ]:
sigma = 3.0

for i, image in enumerate(images):
    fig, axes = pplt.subplots(ncols=2, figsize=(8, 2.5))
    axes[0].pcolormesh(image.xx, image.yy, image.Z.T)
    axes[1].pcolormesh(image.xx, image.yy, image.filter(sigma).T)
    axes[0].set_title('Raw')
    axes[0].set_title(r'Step {}: $\mu_x = {:.2f}\degree$, $\mu_y = {:.2f}\degree$.'.format(
        i, np.degrees(phase_adv[i, 0]), np.degrees(phase_adv[i, 1])))
    axes.format(xlabel='x [mm]', ylabel='y [mm]', aspect=1.0)
    plt.show()